In [1]:
#train3 from 2: use adam, remove weight_decay; one BW channel (not 3)
import torch,os 
import torch.nn as nn
import torch.optim as optim

from torchvision.transforms import v2
import torchvision.models as models
import pandas as pd
import matplotlib.pyplot as plt

# from models import *
from utils import progress_bar
from tqdm import tqdm
from configparser import ConfigParser
from torch.utils.data import  DataLoader
from LIDC_data import LIDC_Dataset

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

prep_tr = [
    v2.Lambda(lambda x: torch.clamp(x,-1000,400)),
    v2.Lambda(lambda x: (x+1000)/1400),
    v2.CenterCrop((384,384)),
    # v2.Lambda(lambda x: x.expand(3,-1,-1))
]
aug_tr = [
    v2.RandomAffine(degrees=10),
    v2.RandomHorizontalFlip(),
    # v2.GaussianNoise(0,0.1)
]
trans_train = v2.Compose( prep_tr + aug_tr )
trans_test = v2.Compose( prep_tr  )

In [3]:

parser = ConfigParser()
parser.read('.settings')
root_dir = parser.get('dataset','root_dir') #/workspaces/data/lidc-idri/slices
meta_dir = parser.get('dataset','meta_dir') #/workspaces/data/lidc-idri/splits
train_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'train_malB.csv'),transform=trans_train)
test_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'test_malB.csv'),transform=trans_test)
total_train_data = len(train_data)
total_test_data = len(test_data)
print('total_train_data:',total_train_data, 'total_test_data:',total_test_data)

trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
testloader = DataLoader(test_data, batch_size=16)

total_train_data: 5495 total_test_data: 2354


In [4]:
# net = ResNet18(num_classes=2)
net = models.resnet18(pretrained=True)
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
net.fc = nn.Linear(net.fc.in_features, 1)
net = net.to(device)

/home/vscode/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vscode/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
lr = 1e-4
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

trainning_accuracy=[]
trainning_loss=[]
testing_accuracy=[]
testing_loss=[]

In [6]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader,desc=f"[Epoch {epoch+1}]")):
        # inputs, targets = inputs.to(device), targets.to(device)
        # targets = targets.float().unsqueeze(1)
        inputs = inputs.to(device)
        targets = targets.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        
        loss = criterion(outputs, targets)
        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()
        # _, predicted = outputs.max(1)

        preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
        
        total += targets.size(0)
        # correct += predicted.eq(targets).sum().item()
        correct += (preds == targets.squeeze().long()).sum().item()

    train_acc = 100.*correct/total
    train_loss = train_loss/(batch_idx+1)
    print(f"Train Loss: {train_loss}, Train Acc: {train_acc:.2f}%")
    trainning_accuracy.append(train_acc)
    trainning_loss.append( train_loss )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            # inputs, targets = inputs.to(device), targets.to(device)
            # targets = targets.float().unsqueeze(1)
            inputs = inputs.to(device)
            targets = targets.float().unsqueeze(1).to(device)

            outputs = net(inputs)
            
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            # _, predicted = outputs.max(1)
            preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
            total += targets.size(0)
            # correct += predicted.eq(targets).sum().item()
            correct += (preds == targets.squeeze().long()).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_acc = 100.*correct/total
        test_loss = test_loss/(batch_idx+1)
        print(f"Test Loss: {test_loss}, Test Acc: {test_acc:.2f}%")
        testing_accuracy.append(100.*correct/total)
        testing_loss.append(test_loss/(batch_idx+1))
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt_owndata.pth')
        best_acc = acc

In [7]:
for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)
 
 #   scheduler.step()

[Epoch 1]: 100%|██████████| 344/344 [03:45<00:00,  1.52it/s]


Train Loss: 0.6760581736994344, Train Acc: 58.56%
 [=================================================>]  Step: 34ms | Tot: 30s670ms | Loss: 0.685 | Acc: 59.176% (1393/2354 148/148  8 =====================================>............]  Step: 202ms | Tot: 23s269ms | Loss: 0.686 | Acc: 59.516% (1057/1776 111/148 
Test Loss: 0.6848937045480754, Test Acc: 59.18%
Saving..


[Epoch 2]: 100%|██████████| 344/344 [01:13<00:00,  4.65it/s]

Train Loss: 0.6260248411360175, Train Acc: 65.17%


 [=================================================>]  Step: 18ms | Tot: 19s812ms | Loss: 0.618 | Acc: 65.718% (1547/2354 148/148  8 
Test Loss: 0.6180919447057956, Test Acc: 65.72%
Saving..


...

[Epoch 4]: 100%|██████████| 344/344 [01:16<00:00,  4.50it/s]

Train Loss: 0.5429344718279534, Train Acc: 72.61%


 [=================================================>]  Step: 17ms | Tot: 17s994ms | Loss: 0.573 | Acc: 70.688% (1664/2354 148/148  8 
Test Loss: 0.5733520863829432, Test Acc: 70.69%


[Epoch 5]:   6%|▌         | 19/344 [00:04<01:10,  4.63it/s]


KeyboardInterrupt: 

In [8]:
targets

NameError: name 'targets' is not defined